In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import matplotlib.pyplot as plt
from PIL import Image

In [4]:
# open method used to open different extension image file
image = Image.open(r"../input/raja-ravi-painting/download (1).jpg") 

imgplot = plt.imshow(image)
plt.show()

cols,rows = image.size

In [5]:
pic = np.array(image.getdata()).reshape(image.size[1], image.size[0], 3)
#pic = np.expand_dims(pic, axis=0)
print(pic[:,:].shape)

#scaling the picture.
pic = pic / 255

In [6]:
print(rows,cols)

In [7]:
#kernels
sobel_vertical = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
sobel_horizontal = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

In [8]:
#rectified linear activation unit. 
def relu(x): 
    if x > 0:
        return x
    else:
        return 0

In [9]:
#computes local W*X
def dot(channel,kernel,i,j):

    sum = (kernel[0,2]*channel[i-1,j+1] + 
            kernel[2,2]*channel[i+1,j+1] + 
            kernel[1,2]*channel[i,j+1] + 
            kernel[0,0]*channel[i-1,j-1] +
            kernel[2,0]*channel[i+1,j-1] +
            kernel[1,0]*channel[i,j-1] +
            kernel[0,1]*channel[i-1,j] + 
            kernel[1,1]*channel[i,j] +
            kernel[2,1]*channel[i+1,j])
        
    return sum
    

In [10]:
#performs convolution seperately for RGB channels and later compines them as a tensor.
def convolution(pic,kernel):
    
    rows,cols,_ = pic[:,:,:].shape
    
    #for red channel
    redChannel = pic[:,:,0]
    actMap_redChannel = np.zeros([rows, cols])
    
    #for green channel
    greenChannel = pic[:,:,1]
    actMap_greenChannel = np.zeros([rows, cols])
    
    #for blue channel
    blueChannel = pic[:,:,2]
    actMap_blueChannel = np.zeros([rows, cols])
    
    for i in range(1,rows-1,1):
        for j in range(1,cols-1,1):
            
            sumR = relu(dot(redChannel,kernel,i,j))
            sumG = relu(dot(greenChannel,kernel,i,j))
            sumB = relu(dot(blueChannel,kernel,i,j))

            actMap_redChannel[i,j] = sumR
            actMap_greenChannel[i,j] = sumG
            actMap_blueChannel[i,j] = sumB
    
    actMap = np.stack((actMap_redChannel,actMap_greenChannel,actMap_blueChannel), axis=2)
    return actMap
    

In [18]:
actMap_vertical_edge = convolution(pic,sobel_vertical)
actMap_horizontal_edge = convolution(pic,sobel_horizontal)

In [19]:
f,ax = plt.subplots(1,2,figsize=(10,10))

ax[0].imshow(actMap_vertical_edge)
ax[0].set_title('vertical edges')

ax[1].imshow(actMap_horizontal_edge)
ax[1].set_title('horizontal edges')

In [24]:
actMap_vertical_edge2 = convolution(actMap_vertical_edge,sobel_vertical)
actMap_horizontal_edge2 = convolution(actMap_horizontal_edge,sobel_horizontal)

In [25]:
f,ax = plt.subplots(1,2,figsize=(10,10))

ax[0].imshow(actMap_vertical_edge2)
ax[0].set_title('vertical edges')

ax[1].imshow(actMap_horizontal_edge2)
ax[1].set_title('horizontal edges')

In [26]:
actMap_horizontal_edge3 = convolution(actMap_vertical_edge2,sobel_horizontal)
actMap_vertical_edge3 = convolution(actMap_horizontal_edge2,sobel_vertical)

In [27]:
f,ax = plt.subplots(1,2,figsize=(10,10))

ax[0].imshow(actMap_vertical_edge3)
ax[0].set_title('vertical edges')

ax[1].imshow(actMap_horizontal_edge3)
ax[1].set_title('horizontal edges')